<a href="https://colab.research.google.com/github/kridtapon/Elder-Ray-Power-Strategy/blob/main/Elder_Ray_Power_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.5/527.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.4 MB/s eta 0:00:00


In [29]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate the Accumulation/Distribution (A/D) line
def calculate_ad_line(data):
    hlc3 = (data['High'] + data['Low'] + data['Close']) / 3  # Typical price
    mfv = (hlc3 - data['Low']) - (data['High'] - hlc3)  # Money Flow Volume
    mfv = mfv / (data['High'] - data['Low']) * data['Volume']  # Money Flow Volume
    ad_line = mfv.cumsum()  # Cumulative sum for A/D line
    return ad_line

# Function to calculate Elder Ray Index
def calculate_elder_ray(data, period=13):
    ema = data['Close'].ewm(span=period, adjust=False).mean()
    bull_power = data['High'] - ema
    bear_power = data['Low'] - ema
    return bull_power, bear_power

# Define the stock symbol and time period
symbol = 'AXON' #AXON GDDY
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Calculate Accumulation/Distribution line
df['AD'] = calculate_ad_line(df)

# Calculate Elder Ray (Bull Power and Bear Power)
df['BullPower'], df['BearPower'] = calculate_elder_ray(df)

# Define Entry and Exit signals based on the conditions
df['Entry'] = (df['AD'] > df['AD'].shift()) & (df['BullPower'] > 0)
df['Exit'] = (df['AD'] < df['AD'].shift()) & (df['BearPower'] > 0)

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed


Start                               2020-01-01 00:00:00
End                                 2024-12-31 00:00:00
Period                               1827 days 00:00:00
Start Value                                    100000.0
End Value                                 970734.919139
Total Return [%]                             870.734919
Benchmark Return [%]                        2447.768145
Max Gross Exposure [%]                            100.0
Total Fees Paid                           318996.107085
Max Drawdown [%]                              78.732596
Max Drawdown Duration                1330 days 00:00:00
Total Trades                                        212
Total Closed Trades                                 212
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                       50.0
Best Trade [%]                                62.705361
Worst Trade [%]                              -60

In [32]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
from sklearn.model_selection import ParameterGrid

# Function to calculate the Accumulation/Distribution (A/D) line
def calculate_ad_line(data):
    hlc3 = (data['High'] + data['Low'] + data['Close']) / 3  # Typical price
    mfv = (hlc3 - data['Low']) - (data['High'] - hlc3)  # Money Flow Volume
    mfv = mfv / (data['High'] - data['Low']) * data['Volume']  # Money Flow Volume
    ad_line = mfv.cumsum()  # Cumulative sum for A/D line
    return ad_line

# Function to calculate Elder Ray Index
def calculate_elder_ray(data, period=13):
    ema = data['Close'].ewm(span=period, adjust=False).mean()
    bull_power = data['High'] - ema
    bear_power = data['Low'] - ema
    return bull_power, bear_power

# Define the stock symbol and time period
symbol = 'GDDY' #AXON GDDY
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Calculate Accumulation/Distribution line
df['AD'] = calculate_ad_line(df)

# Define grid search parameters for Elder Ray period and thresholds
param_grid = {
    'period': range(5, 31),
    'bull_threshold': [0, 0.5, 1],
    'bear_threshold': [0, -0.5, -1]
}

# Generate all combinations of parameters
grid = ParameterGrid(param_grid)

# Initialize the best performance variables
best_performance = -np.inf
best_params = {}

# Iterate over each parameter combination and backtest
for params in grid:
    # Calculate Elder Ray (Bull Power and Bear Power) with the current parameters
    df['BullPower'], df['BearPower'] = calculate_elder_ray(df, period=params['period'])

    # Define Entry and Exit signals based on the conditions
    df['Entry'] = (df['AD'] > df['AD'].shift()) & (df['BullPower'] > params['bull_threshold'])
    df['Exit'] = (df['AD'] < df['AD'].shift()) & (df['BearPower'] < params['bear_threshold'])

    # Filter data for the test period (2020-2025)
    df_test = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

    # Backtest using vectorbt
    portfolio = vbt.Portfolio.from_signals(
        close=df_test['Close'],
        entries=df_test['Entry'],
        exits=df_test['Exit'],
        init_cash=100_000,
        fees=0.001
    )

    # Evaluate performance (total return in this case)
    performance = portfolio.total_return()

    # Update best performance if the current combination outperforms previous ones
    if performance > best_performance:
        best_performance = performance
        best_params = params

# Print the best parameters and corresponding performance
print(f"Best Parameters: {best_params}")
print(f"Best Performance (Total Return): {best_performance}")

# Recalculate the Elder Ray using the best parameters
df['BullPower'], df['BearPower'] = calculate_elder_ray(df, period=best_params['period'])
df['Entry'] = (df['AD'] > df['AD'].shift()) & (df['BullPower'] > best_params['bull_threshold'])
df['Exit'] = (df['AD'] < df['AD'].shift()) & (df['BearPower'] < best_params['bear_threshold'])

# Filter data for the test period (2020-2025)
df_test = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Final backtest using the best parameters
portfolio = vbt.Portfolio.from_signals(
    close=df_test['Close'],
    entries=df_test['Entry'],
    exits=df_test['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display final performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed


Best Parameters: {'bear_threshold': -1, 'bull_threshold': 0, 'period': 6}
Best Performance (Total Return): 1.0336242973675394
Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           203362.429737
Total Return [%]                        103.36243
Benchmark Return [%]                   187.879233
Max Gross Exposure [%]                      100.0
Total Fees Paid                      27895.040524
Max Drawdown [%]                        39.534742
Max Drawdown Duration                       737.0
Total Trades                                  104
Total Closed Trades                           104
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            42.307692
Best Trade [%]                          40.412079
Worst Trade [%]         

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set

